## Stroke Probability Prediction

## Setting up Spark and getting data

In [3]:
import os
from pyspark.sql import SparkSession
import pyspark.sql as sparksql

## Exploration of data

In [4]:
file_path = os.path.join('.', 'dataset', 'train_2v.csv')
spark = SparkSession.builder.appName('stroke').getOrCreate()
train = spark.read.csv(file_path, inferSchema=True,header=True)

In [6]:
train.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [8]:
train.describe()

DataFrame[summary: string, id: string, gender: string, age: string, hypertension: string, heart_disease: string, ever_married: string, work_type: string, Residence_type: string, avg_glucose_level: string, bmi: string, smoking_status: string, stroke: string]

In [10]:
train.groupBy('stroke').count().show()


+------+-----+
|stroke|count|
+------+-----+
|     1|  783|
|     0|42617|
+------+-----+



In [11]:
train.createOrReplaceTempView('table')

## Starting with Analysis using queries

In [14]:
spark.sql("SELECT work_type, count(work_type) as work_type_count FROM table WHERE stroke == 1 GROUP BY work_type ORDER BY work_type_count DESC").show()

+-------------+---------------+
|    work_type|work_type_count|
+-------------+---------------+
|      Private|            441|
|Self-employed|            251|
|     Govt_job|             89|
|     children|              2|
+-------------+---------------+



## From above results, it looks like private ocupation is the most dangerous

In [15]:
spark.sql("SELECT gender, count(gender) as count_gender, count(gender)*100/sum(count(gender)) over() as percent FROM table GROUP BY gender").show()

+------+------------+-------------------+
|gender|count_gender|            percent|
+------+------------+-------------------+
|Female|       25665|  59.13594470046083|
| Other|          11|0.02534562211981567|
|  Male|       17724|  40.83870967741935|
+------+------------+-------------------+



## To retrieve information about how many Female/Male have a stroke:

In [16]:
spark.sql("SELECT gender, count(gender), (COUNT(gender) * 100.0) /(SELECT count(gender) FROM table WHERE gender == 'Male') as percentage FROM table WHERE stroke = '1' and gender = 'Male' GROUP BY gender").show()

+------+-------------+----------------+
|gender|count(gender)|      percentage|
+------+-------------+----------------+
|  Male|          352|1.98600767321146|
+------+-------------+----------------+



In [17]:
spark.sql("SELECT gender, count(gender), (COUNT(gender) * 100.0) /(SELECT count(gender) FROM table WHERE gender == 'Female') as percentage FROM table WHERE stroke = '1' and gender = 'Female' GROUP BY gender").show()

+------+-------------+----------------+
|gender|count(gender)|      percentage|
+------+-------------+----------------+
|Female|          431|1.67932982661212|
+------+-------------+----------------+



##  To see if the age has an influence on stroke and what is the risk by age.

In [19]:
spark.sql("SELECT age, count(age) as age_count FROM table WHERE stroke == 1 GROUP BY age ORDER BY age_count DESC").show()


+----+---------+
| age|age_count|
+----+---------+
|79.0|       70|
|78.0|       57|
|80.0|       49|
|81.0|       43|
|82.0|       36|
|70.0|       25|
|74.0|       24|
|77.0|       24|
|76.0|       24|
|75.0|       23|
|67.0|       23|
|72.0|       21|
|69.0|       20|
|68.0|       20|
|59.0|       20|
|71.0|       19|
|57.0|       19|
|63.0|       18|
|65.0|       18|
|66.0|       17|
+----+---------+
only showing top 20 rows



In [20]:
train.filter((train['stroke'] == 1) & (train['age'] > '50')).count()


708

## Cleaning data
The next step of exploration is to deal with categorical and missing values. There are missing values for smoking_status and bmi parameters. Fill out smoking_status with a value of ‘No info’ and bmi parameter with mean value.

In [22]:

train_f = train.na.fill('No Info', subset=['smoking_status'])

from pyspark.sql.functions import mean
mean = train_f.select(mean(train_f['bmi'])).collect()
mean_bmi = mean[0][0]
train_f = train_f.na.fill(mean_bmi,['bmi'])

## Most of ML algorithms cannot work directly with categorical data. The encoding allows algorithms which expect continuous features to use categorical features.

In [23]:
from pyspark.ml.feature import (VectorAssembler,OneHotEncoder,
                                StringIndexer)

In [48]:
gender_indexer=StringIndexer(inputCol='gender',outputCol='genderIndex' )
gender_encoder= OneHotEncoder(inputCol='genderIndex', outputCol='genderVec')


In [49]:
ever_married_indexer=StringIndexer(inputCol='ever_married',outputCol='ever_marriedIndex' )
ever_married_encoder= OneHotEncoder(inputCol='ever_marriedIndex', outputCol='ever_marriedVec')

In [50]:
Residence_type_indexer=StringIndexer(inputCol='Residence_type',outputCol='Residence_typeIndex' )
Residence_type_encoder= OneHotEncoder(inputCol='Residence_typeIndex', outputCol='Residence_typeVec')

In [63]:
work_type_indexer=StringIndexer(inputCol='work_type',outputCol='work_typeIndex' )
work_type_encoder= OneHotEncoder(inputCol='work_typeIndex', outputCol='work_typeVec')

In [64]:
smoking_status_indexer=StringIndexer(inputCol='smoking_status',outputCol='smoking_statusIndex' )
smoking_status_encoder= OneHotEncoder(inputCol='smoking_statusIndex', outputCol='smoking_statusVec')

## To create an assembler, that combines a given list of columns into a single vector column to train ML model.

In [65]:
assembler = VectorAssembler(inputCols=['genderVec',
 'age',
 'hypertension',
 'heart_disease',
 'ever_marriedVec',
 'work_typeVec',
 'Residence_typeVec',
 'avg_glucose_level',
 'bmi',
 'smoking_statusVec'],outputCol='features')

## Create a DecisionTree object

In [66]:
from pyspark.ml.classification import DecisionTreeClassifier
dtc = DecisionTreeClassifier(labelCol='stroke',featuresCol='features')

## Sequence of PipelineStages 

In [67]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[gender_indexer, ever_married_indexer, work_type_indexer, Residence_type_indexer,
                           smoking_status_indexer, gender_encoder, ever_married_encoder, work_type_encoder,
                           Residence_type_encoder, smoking_status_encoder, assembler, dtc])

## Split dataset to train and test data

In [68]:
train_data,test_data = train_f.randomSplit([0.7,0.3])

## Fit the model

In [69]:
model = pipeline.fit(train_data)

## Transform the test_data.

In [70]:
dtc_predictions = model.transform(test_data)

## Evaluate the model

In [71]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Select (prediction, true label) and compute test error
acc_evaluator = MulticlassClassificationEvaluator(labelCol="stroke", predictionCol="prediction", metricName="accuracy")
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
print('A Decision Tree algorithm had an accuracy of: {0:2.2f}%'.format(dtc_acc*100))

A Decision Tree algorithm had an accuracy of: 98.26%


## Thus, the decesion tree model we implemented using PySpark had an accuracy of 98.26%